# 1.5 - Macrobond Data API - In-house Series - Uploading and Fetching

This notebook is designed to act as a template and guidline in which certain elements can be minipulated to get the desired outcome.

Here we demonstrate how to upload, fetch and delete in-house series. In-house series are proprietary content you own and wish to import in Macrobond for further analysis.

*The examples use the common functions of Macrobond API. Full error handling is omitted for brevity*

***

## Importing packages

In [0]:
import pandas as pd
import macrobond_data_api as mda
import numpy as np

pd.options.display.max_colwidth = 1000

***

## Get and process the data

In this example, instead of starting with a blank series, we first retrieve an existing time series from the Macrobond database, that will then be altered before imported back into the Macrobond ecosystem.

In [0]:
# get a timeseries
series = mda.get_one_series("ustrad2120")
metadata = series.metadata
series = series.values_to_pd_data_frame()
series.index = pd.to_datetime(series["date"])
series = series.drop(columns=["date"]).rename(columns={"value": "ustrad2120"})
series.plot()

We are now going to perform some alterations on the time series.

In [0]:
# process the timeseries, in this case, smooth it by moving average
padded_data = np.pad(series["ustrad2120"], (48 // 2, 48 // 2), mode="reflect")
trend = np.convolve(padded_data, np.ones(48), "valid") / 48
series_smooth = pd.DataFrame({"value": trend[-len(series) :]}, index=series.index)
series_smooth.rename(columns={"value": "ustrad2120_smooth"}, inplace=True)
series_smooth.plot()

In [0]:
series_smooth

***

## Uploading processed series to Macrobond as in-house series

Now, you can upload your processed series to Macrobond as in-house series. You have three directories where you can save your series in Macrobond.
* Private Account: account belonging to the API account used to perform this upload. No one else could access this in-house series.
* Department Account: All the users in the same department with this account can see and fetch it.
* Company Account: All the users in the same company with this account can see and fetch it.

Note: If you are uploading a time series using a Data Web API data feed account, you need to upload the series to department account or company account so you and your team's personal Macrobond accounts, including the Macrobond Application (if any) can see and fetch it.

In [0]:
from macrobond_data_api.common.enums.series_frequency import SeriesFrequency
from datetime import datetime
from dateutil.tz import tzutc

mda.upload_series(
    name="ih:mb:com:ustrad2120_smooth",
    description="{} Smoothed".format(metadata["Description"]),
    category="{}_smoothed".format(metadata["Category"]),
    region=metadata["Region"][0],
    frequency=SeriesFrequency.WEEKLY,
    metadata={
        "Source": metadata["Source"],
    },  # metadata added here has to be metadata known by Macrobond, not custom metadata
    values=series_smooth["{}_smooth".format(metadata["PrimName"])],
    start_date_or_dates=series_smooth.index.tz_localize(tzutc()),
)  # the dates must have timezone information

***

## Fetch in house series
You can fetch any in-house series saved in personal account by yourself or in department/company account by you or your colleagues in the same department/company, just like fetching any Macrobond time series. The fetching methods being the same, only the primname of the request is specific to your in-house series.

In [0]:
mda.get_one_series("ih:mb:com:ustrad2120_smooth")

***

## Delete in house series

In [0]:
mda.delete_serie("ih:mb:com:ustrad2120_smooth")